<a href="https://colab.research.google.com/github/eanorambuena/2024_web/blob/main/T1_Coverage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 1: Parte A

## Clase Coverage

Tome de base la clase Coverage del libro fuzzing-book. La tarea consiste en mejorar esta clase para que el reporte de coverage se vea a nivel de clase y método. Es decir debe decir que porcentaje de cada clase fue cubierto o porcentaje de cada método.

In [195]:
import sys
import inspect
from types import FrameType, TracebackType
from typing import Any, Callable, List, Optional, Set, Tuple, Type, Dict

Location = Tuple[str, int]

class Coverage:
    """Track coverage within a `with` block. Use as
    ```
    with Coverage() as cov:
        function_to_be_traced()
    c = cov.coverage()
    ```
    """

    def __init__(self, *classes_to_check) -> None:
        """Constructor"""
        self._trace: List[Location] = []
        self._classes_to_check = classes_to_check

    # Trace function
    def traceit(self, frame: FrameType, event: str, arg: Any) -> Optional[Callable]:
        """Tracing function. To be overloaded in subclasses."""
        if self.original_trace_function is not None:
            self.original_trace_function(frame, event, arg)

        if event == "line":
            function_name = frame.f_code.co_name
            lineno = frame.f_lineno
            if function_name != '__exit__':  # avoid tracing ourselves:
                self._trace.append((function_name, lineno))

        return self.traceit

    def __enter__(self) -> Any:
        """Start of `with` block. Turn on tracing."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self.traceit)
        return self

    def __exit__(self, exc_type: Type, exc_value: BaseException,
                 tb: TracebackType) -> Optional[bool]:
        """End of `with` block. Turn off tracing."""
        sys.settrace(self.original_trace_function)
        return None  # default: pass all exceptions

    def trace(self) -> List[Location]:
        """The list of executed lines, as (function_name, line_number) pairs"""
        return self._trace

    def coverage(self) -> Set[Location]:
        """The set of executed lines, as (function_name, line_number) pairs"""
        return set(self.trace())

    def function_names(self) -> Set[str]:
        """The set of function names seen"""
        return set(function_name for (function_name, line_number) in self.coverage())

    def report(self):
        """Returns the formatted report string"""
        covered_functions = self.function_names()
        class_coverage_outputs = []

        for cls in self._classes_to_check:
            method_coverage_outputs = []
            cls_method_names = [name for name, _ in inspect.getmembers(cls, predicate=inspect.isfunction) if not name.startswith('__')]
            number_of_covered_methods = 0
            for method in cls_method_names:
                is_method_covered = method in covered_functions
                method_coverage_outputs.append(f"\t{method}: {is_method_covered}")
                if is_method_covered:
                  number_of_covered_methods += 1
            class_coverage_outputs.append(f"Class {cls.__name__}: {number_of_covered_methods/len(cls_method_names)*100}%\n{'\n'.join(method_coverage_outputs)}")

        return "\n".join(class_coverage_outputs)

## Caso de Estudio 1: Clock y User Session




### Run Test Method

In [196]:
def runTests(testClass):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(testClass)
    runner = unittest.TextTestRunner()
    runner.run(suite)

### Clases Under Tests

In [197]:
from datetime import timedelta
from datetime import datetime

class Clock:
    count = 0
    def __init__(self):
        Clock.count += 1

    def now(self):
        return datetime.now()

    @classmethod
    def get_instance_count(cls):
        return cls.count

class UserSession:
    def __init__(self, user_id: str, clock: Clock = None):
        self.user_id = user_id
        self.logged_in = False
        self.login_time = None
        self.failed_attempts = 0
        self.clock = clock or Clock()

    def login(self, password: str) -> None:
        if self.logged_in:
            raise ValueError("User already logged in")
        if not self._check_password(password):
            self.failed_attempts += 1
            if self.failed_attempts >= 3:
                raise ValueError("Too many failed attempts")
            raise ValueError("Invalid password")
        self.logged_in = True
        self.login_time = self.clock.now()
        self.failed_attempts = 0

    def logout(self) -> None:
        if not self.logged_in:
            raise ValueError("User not logged in")
        self.logged_in = False
        self.login_time = None

    def is_session_active(self) -> bool:
        if not self.logged_in:
            return False
        if self.clock.now() - self.login_time > timedelta(minutes=30):
            self.logged_in = False
            return False
        return True

    def _check_password(self, password: str) -> bool:
        return password == "secure123"

### Unit Tests

In [198]:
import unittest
from unittest.mock import Mock
from datetime import datetime, timedelta

class TestUserSession(unittest.TestCase):
    def test_successful_login(self):
        self.start_time = datetime(2023, 1, 1, 12, 0, 0)
        self.clock = Mock()
        self.clock.now.return_value = self.start_time
        self.session = UserSession("user123", clock=self.clock)
        self.session.login("secure123")
        self.assertTrue(self.session.logged_in)
        self.assertEqual(self.session.login_time, self.start_time)
        self.assertEqual(self.session.failed_attempts, 0)
        self.assertTrue(self.session.is_session_active())

    def test_not_successful_login(self):
        self.start_time = datetime(2023, 1, 1, 12, 0, 0)
        self.clock = Mock()
        self.clock.now.return_value = self.start_time
        self.session = UserSession("user123", clock=self.clock)
        with self.assertRaises(ValueError) as cm:
            self.session.login("secure1234")
        self.assertEqual(str(cm.exception), "Invalid password")
        self.assertFalse(self.session.is_session_active())

### Prueba 1: Coverage Report

Modifique la calse Coverage Report para que reciba como argumento las clases de las cuales se quiere analizar la covertura. Ademas agregue un método report que devuelve un string formateado con el reporte.

In [199]:
with Coverage(UserSession, Clock) as cov:
    runTests(TestUserSession)

print(cov.report())

..
----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


Class UserSession: 75.0%
	_check_password: True
	is_session_active: True
	login: True
	logout: False
Class Clock: 0.0%
	now: False


## Caso Estudio 2: Escriba un Segundo Caso de Estudio para probar que su solución
Esto demostrar que su solución funciona para analizar cualquier clase y no solo las del caso de estudio 1.

### Units Under Tests

In [200]:
## Ingrese tres clases que colaboran entre si
class Persona:
    def __init__(self, nombre):
        self._nombre = nombre

    def obtener_nombre(self):
        return self._nombre

    def obtener_pais(self):
        return "Chile"

class Apuesta:
    def __init__(self, objeto, accion):
        self._objeto = objeto
        self._accion = accion

    def obtener_apuesta(self):
        return f"te apuesto un {self._objeto} a que {self._accion}"

class Saludador:
    def obtener_saludo(self, persona, apuesta):
        return f"Hola soy {persona.obtener_nombre()} y {apuesta.obtener_apuesta}"

### Unit Tests

In [201]:
## Ingrese unit test para las clases anteriores
import unittest
from unittest.mock import Mock
from datetime import datetime, timedelta

class TestPersona(unittest.TestCase):
    def test_successful_saludar(self):
        persona = Persona("Germán")
        self.assertEqual(persona.obtener_nombre(), "Germán")

class TestApuesta(unittest.TestCase):
    def test_successful_apuesta(self):
        apuesta = Apuesta("naranja", "duermes con los ojos cerrados")
        self.assertGreater(len(apuesta.obtener_apuesta()), 0)

class TestSaludador(unittest.TestCase):
    def test_successful_saludar(self):
        persona = Persona("Germán")
        apuesta = Apuesta("naranja", "duermes con los ojos cerrados")
        saludador = Saludador()
        self.assertGreater(len(saludador.obtener_saludo(persona, apuesta)), 0)

### Prueba 2: Coverage Report

In [202]:
# Escriba un script para analizar el coverage del caso de estudio 2

# with Coverage( <clases under test> ) as cov:
#    runTests(<test class>)
#    runTests(<test class>)
#    runTests(<test class>)

# print(cov.report())

with Coverage(Persona, Apuesta, Saludador) as cov:
    runTests(TestPersona)
    runTests(TestApuesta)
    runTests(TestSaludador)

print(cov.report())

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


Class Persona: 50.0%
	obtener_nombre: True
	obtener_pais: False
Class Apuesta: 100.0%
	obtener_apuesta: True
Class Saludador: 100.0%
	obtener_saludo: True
